# SQLite databases

This script migrates scraped data from csv to sqlite databases.

# Load modules

In [ ]:
import pandas as pd
import sqlite3

# Create tables

## Papers

In [ ]:
conn = sqlite3.connect("../recerca.db")
c = conn.cursor()

c.execute("DROP TABLE papers")
c.execute("""CREATE TABLE papers (
          id INTEGER PRIMARY KEY,
          url TEXT,
          url_stem TEXT,
          date TEXT,
          publisher TEXT,
          title TEXT,
          type TEXT,
          author TEXT,
          sourceid TEXT,
          sourceref TEXT,
          orcids TEXT,
          citation TEXT,
          issn TEXT,
          published_in TEXT,
          doi TEXT,
          isbn TEXT,
          uri TEXT,
          status_code INTEGER,
          status_description TEXT,
          date_created TEXT,
          current INTEGER,
          UNIQUE(url,url_stem,date,publisher,title,type,author, 
          sourceid,sourceref,orcids,citation,issn,published_in,doi, 
          isbn,uri,status_code,status_description)
         )""")

conn.commit()
conn.close()

conn = sqlite3.connect("../recerca.db")
c = conn.cursor()
c.execute("""CREATE UNIQUE INDEX ix ON papers (
          ifnull(url,-1),
          url_stem,
          ifnull(date, -1),
          ifnull(publisher, -1),
          ifnull(title, -1),
          ifnull(type, -1),
          ifnull(author, -1),
          ifnull(sourceid, -1),
          ifnull(sourceref, -1),
          ifnull(orcids, -1),
          ifnull(citation, -1),
          ifnull(issn, -1),
          ifnull(published_in, -1),
          ifnull(doi, -1),
          ifnull(isbn, -1),
          ifnull(uri, -1),
          ifnull(status_code, -1),
          ifnull(status_description, -1) )""")

conn.commit()
conn.close()

## URLs

In [ ]:
conn = sqlite3.connect("../recerca.db")
c = conn.cursor()

# c.execute("DROP TABLE urls")
c.execute("""CREATE TABLE IF NOT EXISTS urls (
          id INTEGER PRIMARY KEY,
          items TEXT,
          url_stem TEXT UNIQUE,
          date_created TEXT,
          current INTEGER,
          url_scraped INTEGER
         )""")

conn.commit()
conn.close()

## Authors

In [ ]:
conn = sqlite3.connect("../recerca.db")
c = conn.cursor()

# c.execute("DROP TABLE authors")
c.execute("""CREATE TABLE authors (
            id TEXT PRIMARY KEY,
            label TEXT,
            department TEXT,
            institution TEXT,
            institution_2 TEXT,
            projects TEXT,
            groups TEXT,
            url TEXT,
            status_description TEXT,
            institution_group TEXT
        )""")

conn.commit()
conn.close()

## Groups

In [ ]:
conn = sqlite3.connect("../recerca.db")
c = conn.cursor()

c.execute("""CREATE TABLE IF NOT EXISTS groups (
          id INTEGER PRIMARY KEY,
          name TEXT,
          institution TEXT,
          group_url TEXT,
          sgr_code TEXT,
          principal_names TEXT,
          principal_ids TEXT,
          researcher_names TEXT,
          researcher_uds TEXT,
          url TEXT,
          url_stem TEXT
         )""")

conn.commit()
conn.close()

## Projects

In [ ]:
conn = sqlite3.connect("../recerca.db")
c = conn.cursor()

# c.execute("DROP TABLE projects")
c.execute("""CREATE TABLE IF NOT EXISTS projects (
          id INTEGER PRIMARY KEY,
          title TEXT,
          official_code TEXT,
          url TEXT,
          start_date TEXT,
          end_date TEXT,
          institution TEXT,
          principal_names TEXT,
          principal_ids TEXT,
          researcher_names TEXT,
          researcher_ids TEXT,
          url_stem TEXT
         )""")

conn.commit()
conn.close()

## Edges

In [ ]:
conn = sqlite3.connect("../recerca.db")
c = conn.cursor()

# c.execute("DROP TABLE edges")
c.execute("""CREATE TABLE IF NOT EXISTS edges (
          id INTEGER PRIMARY KEY,
          Source TEXT,
          Target TEXT,
          Weight INTEGER,
          institution TEXT
         )""")

conn.commit()
conn.close()

## Test

In [ ]:
conn = sqlite3.connect("../recerca.db")
c = conn.cursor()

c.execute("DROP TABLE test")
c.execute("""CREATE TABLE IF NOT EXISTS test (
          id INTEGER PRIMARY KEY,
          url TEXT,
          url_stem TEXT,
          date TEXT,
          publisher TEXT,
          title TEXT,
          type TEXT,
          author TEXT,
          sourceid TEXT,
          sourceref TEXT,
          orcids TEXT,
          citation TEXT,
          issn TEXT,
          published_in TEXT,
          doi TEXT,
          isbn TEXT,
          uri TEXT,
          status_code INTEGER,
          status_description TEXT,
          date_created TEXT,
          current INTEGER,
          url_scraped INTEGER
         )""")

conn.commit()
conn.close()

# Upload to SQLite

## URLS

### Paper URLs

In [ ]:
item_list = ['author', 'paper', 'group', 'project']

for item in item_list:
    print(f"Item: {item}")
    # Load data
    print("Reading csv.")
    df = pd.read_csv(f'../data/20220314/{item}_links_20220314.csv')

    # Rename columns
    renamed_cols = {'0':'url_stem'}
    df = df.rename(columns = renamed_cols)

    # Remove leading backslash
    df['url_stem'] = df['url_stem'].str[1:]

    # Format data
    df = df.sort_values(by='url_stem')
    df = df.drop_duplicates(subset='url_stem')
    df = df.reset_index(drop=True)
#     df = df.reset_index(drop=True).reset_index()
#     df = df.rename(columns={'index': 'id'})

    # New columns
    df['items']= item
    df['date_created'] = '20220314'
    df['url_scraped'] = 1
    df['current'] = 1

    print("Writing to sqlite.")
    # Upload to sqlite
    conn = sqlite3.connect("../recerca.db")
    df.to_sql(name='urls', con = conn, if_exists='append', index=False)
    conn.close()
    print("Done.")

## Papers

In [ ]:
# Load data
df = pd.read_csv('../data/20220314/paper_clean_20220314.csv')

# Rename columns
renamed_cols = {'url_id':'url_stem',
                'status code':'status_code',
                'status description':'status_description'}
df = df.rename(columns = renamed_cols)

# Format data
df = df.sort_values(by='url_stem')
df = df.drop_duplicates(subset='url_stem')
df = df.reset_index(drop=True).reset_index()
df = df.rename(columns={'index': 'id'})

# New columns
df['date_created'] = '20220314'
df['current'] = 1

In [ ]:
### Upload to sqlite
conn = sqlite3.connect("../recerca.db")
df.to_sql(name='papers', con = conn, if_exists='append', index=False)
conn.close()

## Authors

### Process csv

In [ ]:
# Load data
df = pd.read_csv('../data/20220314/author_clean_20220314.csv')

# Rename columns
renamed_cols = {'status description':'status_description'}
df = df.rename(columns = renamed_cols)

### Upload to sqlite

In [ ]:
conn = sqlite3.connect("../recerca.db")
df.to_sql(name='authors', con = conn, if_exists='append', index=False)
conn.close()

### Read data

In [ ]:
# Connection
conn = sqlite3.connect("../recerca.db")
conn.row_factory = sqlite3.Row

# Query
c = conn.cursor()
# sql = "SELECT * FROM sqlite_master WHERE type='table'"
# sql = "SELECT name FROM sqlite_master WHERE type='table'"
# sql = "pragma table_info('papers')"
sql = "SELECT COUNT(*) FROM authors"
# sql = "SELECT * FROM authors"
c.execute(sql)

# Print
result = c.fetchone()
print(dict(result))

conn.commit()
conn.close()

## Groups

### Process csv

In [ ]:
# Load data
df = pd.read_csv('../data/20220314/group_data_20220314.csv')

rename_cols = {
    'principal names' : 'principal_names',
    'principal ids' : 'principal_ids',
    'researcher names' : 'researcher_names',
    'researcher ids' : 'researcher_ids',
    'url_id':'url_stem'
}
df = df.rename(columns=rename_cols)

df = df.sort_values(by='url_stem')
df = df.drop_duplicates(subset='url_stem')
df = df.reset_index(drop=True).reset_index()
df = df.rename(columns={'index': 'id'})

### Upload to sqlite

In [ ]:
conn = sqlite3.connect("../recerca.db")
df.to_sql(name='groups', con = conn, if_exists='append', index=False)
conn.close()

### Read data

In [ ]:
# Connection
conn = sqlite3.connect("../recerca.db")
conn.row_factory = sqlite3.Row

# Query
c = conn.cursor()
# sql = "SELECT * FROM sqlite_master WHERE type='table'"
# sql = "SELECT name FROM sqlite_master WHERE type='table'"
# sql = "pragma table_info('papers')"
# sql = "SELECT COUNT(*) FROM groups"
sql = "SELECT * FROM groups"
c.execute(sql)

# Print
result = c.fetchone()
print(dict(result))

conn.commit()
conn.close()

## Projects

### Process csv

In [ ]:
# Load data
df = pd.read_csv('../data/20220314/project_data_20220314.csv')

rename_cols = {
    'official code' : 'official_code',
    'start date' : 'start_date',
    'end date' : 'end_date',
    'principal names' : 'principal_names',
    'principal ids' : 'principal_ids',
    'researcher names' : 'researcher_names',
    'researcher ids' : 'researcher_ids',
    'url_id' : 'url_stem',
}
df = df.rename(columns=rename_cols)

df = df.sort_values(by='url_stem')
df = df.drop_duplicates(subset='url_stem')
df = df.reset_index(drop=True).reset_index()
df = df.rename(columns={'index': 'id'})

### Upload to sqlite

In [ ]:
conn = sqlite3.connect("../recerca.db")
df.to_sql(name='projects', con = conn, if_exists='append', index=False)
conn.close()

### Read data

In [ ]:
# Connection
conn = sqlite3.connect("../recerca.db")
conn.row_factory = sqlite3.Row

# Query
c = conn.cursor()
# sql = "SELECT * FROM sqlite_master WHERE type='table'"
# sql = "SELECT name FROM sqlite_master WHERE type='table'"
# sql = "pragma table_info('papers')"
# sql = "SELECT COUNT(*) FROM groups"
sql = "SELECT * FROM projects"
c.execute(sql)

# Print
result = c.fetchone()
print(dict(result))

conn.commit()
conn.close()

## Edges

### Process csv

In [ ]:
# Load data
institution_list = ['IGTP+', 'UB', 'UOC', 'UPC_CIMNE', 'UPF', 'UVic-UCC']
for institution in institution_list:
    print(f"Institution: {institution}.")
    df = pd.read_csv(f'../data/20220314/edges_{institution}_20220314.csv')
    df = df.loc[df['Weight']>0]
    df = df.sort_values(by=['Source', 'Target'])
    df = df.drop_duplicates(subset=['Source', 'Target'])
    df = df.reset_index(drop=True)
    df['institution'] = institution
    # Upload to sqlite
    conn = sqlite3.connect("../recerca.db")
    df.to_sql(name='edges', con = conn, if_exists='append', index=False)
    conn.close()

### Read data

In [ ]:
# Connection
conn = sqlite3.connect("../recerca.db")
conn.row_factory = sqlite3.Row

# Query
c = conn.cursor()
# sql = "SELECT * FROM sqlite_master WHERE type='table'"
# sql = "SELECT name FROM sqlite_master WHERE type='table'"
# sql = "pragma table_info('papers')"
# sql = "SELECT COUNT(*) FROM groups"
sql = "SELECT * FROM edges WHERE institution = 'UB'"
c.execute(sql)

# Print
result = c.fetchmany(10)
for item in result:
    print(dict(item))
# print(dict(result))

conn.commit()
conn.close()

# Test scripts

## Test: insert paper into papers table

In [ ]:
def insert_papers(papers, date):
    for paper in papers:
        paper['current'] = 1
        paper['date_created'] = date

    conn = sqlite3.connect('../recerca.db')
    with conn:
    #     c = conn.cursor()

        # Set current = 0 for existing records
        conn.executemany("""UPDATE papers
                        SET current = 0
                        WHERE url_stem = :url_stem""", papers)

        # Insert new record
        conn.executemany("""INSERT INTO papers VALUES (
                        :id,
                        :url,:url_stem,:date,:publisher,:title,:type,:author,
                        :sourceid,:sourceref,:orcids,:citation,:issn,:published_in,
                        :doi,:isbn,:uri,:status_code,:status_description,
                        :date_created,:current) 
                        ON CONFLICT DO UPDATE SET current=1
                        """, papers)

    conn.close()

In [ ]:

                        ON CONFLICT (url, url_stem, date, publisher, title, type, author, 
                          sourceid, sourceref, orcids, citation, issn, published_in, doi, 
                          isbn, uri, status_code, status_description)
                        DO UPDATE SET current = 1

In [ ]:
conn = sqlite3.connect('../recerca.db')
conn.row_factory = sqlite3.Row
c = conn.cursor()

# c.execute("SELECT url_stem FROM test WHERE EXISTS(SELECT 1 FROM test WHERE url_stem = ?) LIMIT 10", 
#           (insert_result[0]['url_stem'],))
# c.execute("SELECT 1 FROM test WHERE url_stem = ?", (insert_result[0]['url_stem'],))
# c.execute("SELECT * FROM papers WHERE date_created = 20220324")
# c.execute("SELECT * FROM papers WHERE current = 1")
c.execute("SELECT * FROM papers ORDER BY id DESC LIMIT 10")
# c.execute("SELECT COUNT(*) FROM papers")

# print(c.fetchall())
result = c.fetchmany(6)
for paper in result:
    print(dict(paper))
    
conn.commit()
conn.close()

In [ ]:
# for key in dict(result[0]).keys():
#     print(key, dict(result[0])[key] == dict(result[3])[key])

In [ ]:
papers = []
for paper in result:
    papers.append(dict(paper))

for paper in papers:
    paper['id'] = None
    paper['title'] = 'NEW NEW TITLE'

In [ ]:
insert_papers(papers, 'XXXXXXXX')

## Test: insert url into paper URLs table

In [ ]:
def insert_urls(urls, date):
    rows = []
    for url in urls:
        row = {}
        row['url_stem'] = url
        row['current'] = 1
        row['url_scraped'] = 0
        row['date_created'] = date
        rows.append(row)

    conn = sqlite3.connect('../recerca.db')
    c = conn.cursor()
    
    c.executemany("""INSERT OR IGNORE INTO paper_urls_test (url_stem, current, url_scraped, date_created) 
              VALUES (:url_stem, :current, :url_scraped, :date_created)""", rows)
        
    conn.commit()
    conn.close()

In [ ]:
urls = ['url1', 'url2', 'url3']

insert_urls(urls, 'XXXXXX')

In [ ]:
conn = sqlite3.connect('../recerca.db')
conn.row_factory = sqlite3.Row
c = conn.cursor()

# c.execute("SELECT url_stem FROM test WHERE EXISTS(SELECT 1 FROM test WHERE url_stem = ?) LIMIT 10", 
#           (insert_result[0]['url_stem'],))
# c.execute("SELECT 1 FROM test WHERE url_stem = ?", (insert_result[0]['url_stem'],))
# c.execute("SELECT * FROM urls ORDER BY id DESC LIMIT 10")
c.execute("SELECT * FROM urls WHERE items = 'group' ORDER BY id DESC LIMIT 10")
# c.execute("SELECT * FROM paper_urls_test WHERE date_created = 'XXXXXX'")
# c.execute("SELECT * FROM papers WHERE current = 1")

# print(c.fetchall())
result = c.fetchmany(10)
for paper in result:
    print(dict(paper))
    
    
conn.commit()
conn.close()

In [ ]:
insert_result[0]

In [ ]:
insert_result[0]

In [ ]:
conn = sqlite3.connect('../recerca.db')
conn.row_factory = sqlite3.Row
c = conn.cursor()

sql = "DROP TABLE deleteme"
sql = """
CREATE TABLE `deleteme` (
    `Field1`    INTEGER,
    `Field2`    INTEGER
);
"""
# sql = """
# CREATE UNIQUE INDEX `ix` ON `deleteme` (
#     `Field1`,
#     `Field2`
# );
# """

sql = """
INSERT INTO `deleteme`(`Field1`,`Field2`) VALUES (1,NULL);
"""
# sql = "SELECT * FROM deleteme"

# sql = """
# INSERT INTO `test`(`Field1`,`Field2`) VALUES (1,NULL); -- This shouldn't be allowed
# """

# sql = """
# DROP INDEX IF EXISTS `ix2`
# """
# sql = """
# CREATE UNIQUE INDEX `ix2` ON `deleteme` (
#     `Field1`,
#     ifnull(`Field2`, 0)  --use this instead
# ); --will fail
# """

c.execute(sql)

# print(c.fetchall())

# result = c.fetchmany(4)

# for paper in result:
#     print(dict(paper))
    
conn.commit()
conn.close()

In [ ]:
print(result)